### **1. Paramètres**

In [1]:
# Importation des bibliothèques
import os
import pandas as pd

In [2]:
# Chemin vers les fichiers
csv_path = '../data_final/tableaux/'

# Lire le fichier CSV
df_pixel = pd.read_csv(os.path.join(csv_path, 'df_pixel.csv'), dtype={'id_parc': str})
df_parcelle = pd.read_csv(os.path.join(csv_path, 'df_parcelle.csv'), dtype={'id_parc': str})

### **2. Fonction pour filtrer les .csv**

In [3]:
def filtrer_dataframe(df, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=False):
    """
    Applique des filtres au DataFrame pour exclure les pixels selon des critères spécifiques.

    Args:
        df (DataFrame): Le DataFrame à filtrer.
        pixels_to_exclude (list): Liste des coordonnées (x, y) à exclure.
        exclude_unique_ids (list): Liste des `unique_id` à exclure complètement.
        criteria (dict): Critères spécifiques pour certains `unique_id`.
        lidar_date_filter (bool): Si True, filtre également selon 'lidar_date' == 'date' et autres conditions Lidar.

    Returns:
        DataFrame: Le DataFrame filtré.
    """
    # Supprimer les pixels selon les coordonnées
    df_filtre = df[~df[["x", "y"]].apply(tuple, axis=1).isin(pixels_to_exclude)]
    
    # Supprimer les pixels selon `unique_id`
    df_filtre = df_filtre[~df_filtre["unique_id"].isin(exclude_unique_ids)]
    
    # Appliquer les critères spécifiques
    for uid, condition in criteria.items():
        df_filtre = df_filtre[~(
            (df_filtre["unique_id"] == uid) &
            (df_filtre.apply(condition, axis=1))
        )]

    # Filtrer selon 'lidar_date' == 'date' et autres conditions spécifiques à Lidar
    if lidar_date_filter:
        # Condition 1: 'lidar_date' == 'date'
        if 'lidar_date' in df_filtre.columns and 'date' in df_filtre.columns:
            df_filtre = df_filtre[df_filtre['lidar_date'] == df_filtre['date']]
        
        # Condition 2: Supprimer où `grid_CC < 5` et `age_plan >= 5`
        if 'grid_CC' in df_filtre.columns and 'age_plan' in df_filtre.columns:
            df_filtre = df_filtre[~((df_filtre['grid_CC'] < 5) & (df_filtre['age_plan'] >= 5))]
        
        # FIltrage dans l'échelle parcelle en utilisant la moyenne du CC (`CC`)
        # if 'CC' in df_filtre.columns and 'age_plan' in df_filtre.columns:
        #     df_filtre = df_filtre[~((df_filtre['CC'] < 5) & (df_filtre['age_plan'] >= 5))]

    return df_filtre


### **3. Filtrage des pixels du Departément 10**

In [4]:
# Définir les paramètres de filtrage
pixels_to_exclude = [
    (736105, 6824467.46), (736115, 6824467.46), (736125, 6824467.46),
    (736785, 6824337.46), (736795, 6824337.46), (736805, 6824337.46)
]

exclude_unique_ids = [
    "dep10_4", "dep10_5", "dep10_6", "dep10_8", "dep10_9", "dep10_14",
    "dep10_17", "dep10_18", "dep10_19", "dep10_20", "dep10_21", "dep10_24",
    "dep10_27", "dep10_29", "dep10_33", "dep10_35", "dep10_38", "dep10_40",
    "dep10_41", "dep10_42", "dep10_45", "dep10_48", "dep10_49", "dep10_50",
    "dep10_51", "dep10_52", "dep10_57", "dep10_59", "dep10_69", "dep10_71",
    "dep10_83", "dep10_87"
]

criteria = {
    "dep10_15": lambda row: row['date'] <= 2019,
    "dep10_65": lambda row: row['date'] <= 2019,
    "dep10_66": lambda row: row['date'] <= 2019,
    "dep10_77": lambda row: row['date'] <= 2019
}


### **3.1 Filtrage des pixels dans le .csv issue des raster Lidar (échelle pixel)**

In [5]:
# Appliquer le filtrage au DataFrame des pixels
df_pixel_filtre_dept10 = filtrer_dataframe(df_pixel, pixels_to_exclude, exclude_unique_ids, criteria)
print(f"Quantité initiale (df_pixel) : {len(df_pixel)}")
print(f"Quantité après filtrage (df_px_filtre_dept10) : {len(df_pixel_filtre_dept10)}")

# Sauvegarder le DataFrame filtré
df_pixel_filtre_dept10.to_csv(os.path.join(csv_path, 'df_pixel_filtre_dept10.csv'), index=False)
print(f"Fichier df_px_filtre_dept10 sauvegardé avec succès.")

Quantité initiale (df_pixel) : 77617
Quantité après filtrage (df_px_filtre_dept10) : 71453
Fichier df_px_filtre_dept10 sauvegardé avec succès.


### **3.2 Filtrage des pixels dans le .csv issue des métriques calculées pour les parcelles (échelle parcelle)**

In [6]:
# Appliquer le filtrage au DataFrame des parcelles
df_parcelle_filtre_dept10 = filtrer_dataframe(df_parcelle, pixels_to_exclude, exclude_unique_ids, criteria)
print(f"Quantité initiale (df_parcelle) : {len(df_parcelle)}")
print(f"Quantité après filtrage (df_parcelle_filtre_dept10) : {len(df_parcelle_filtre_dept10)}")

# Sauvegarder le DataFrame filtré
df_parcelle_filtre_dept10.to_csv(os.path.join(csv_path, 'df_parcelle_filtre_dept10.csv'), index=False)
print(f"Fichier df_parcelle_filtre_dept10 sauvegardé avec succès.")

Quantité initiale (df_parcelle) : 77617
Quantité après filtrage (df_parcelle_filtre_dept10) : 71453
Fichier df_parcelle_filtre_dept10 sauvegardé avec succès.


### **4. Filtrage pour les métriques lidar**

### **4.1 Filtrage pour les métriques lidar dans le .csv issue des raster Lidar (échelle pixel)**

In [7]:

# Filtrer selon 'lidar_date' == 'date' pour les pixels
df_pixel_filtre_lidar = filtrer_dataframe(df_pixel, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=True)
print(f"Quantité après filtrage lidar (df_px_filtre_lidar) : {len(df_pixel_filtre_lidar)}")

# Sauvegarder le DataFrame filtré
df_pixel_filtre_lidar.to_csv(os.path.join(csv_path, 'df_pixel_filtre_lidar.csv'), index=False)
print(f"Fichier df_px_filtre_lidar sauvegardé avec succès.")

Quantité après filtrage lidar (df_px_filtre_lidar) : 9031
Fichier df_px_filtre_lidar sauvegardé avec succès.


### **4.2 Filtrage pour les métriques lidar dans le .csv issue des métriques calculées pour les parcelles (échelle parcelle)**

In [8]:
# Filtrer selon 'lidar_date' == 'date' pour les parcelles
df_parcelle_filtre_lidar = filtrer_dataframe(df_parcelle, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=True)
print(f"Quantité après filtrage lidar (df_parcelle_filtre_lidar) : {len(df_parcelle_filtre_lidar)}")

# Sauvegarder le DataFrame filtré
df_parcelle_filtre_lidar.to_csv(os.path.join(csv_path, 'df_parcelle_filtre_lidar.csv'), index=False)
print(f"Fichier df_parcelle_filtre_lidar sauvegardé avec succès.")

Quantité après filtrage lidar (df_parcelle_filtre_lidar) : 10111
Fichier df_parcelle_filtre_lidar sauvegardé avec succès.
